In [1]:
import psycopg2 as pg2
import json

try:
    with open('db_credentials.json', 'r') as f:
        data = json.load(f)
        USER = data['username']
        PASSWORD = data['password']

except FileNotFoundError as e:
    print('Error: Invalid file name.')

except KeyError as e:
    print('Error: Invalid Key in json file.') 

except Exception as e:
    print('Error: Invalid workflow.')
    print(e)


In [2]:
try:
    conn = pg2.connect(
        host='localhost',
        database='udacity',
        user=USER,
        password=PASSWORD
    )
    conn.set_session(autocommit=True)
    cur = conn.cursor()

except Exception as e:
    print(e)

In [26]:
query = "DROP TABLE IF EXISTS music_store"
cur.execute(query)

In [27]:
query = """
CREATE TABLE IF NOT EXISTS music_store (
    transaction_id INT NOT NULL UNIQUE,
    customer_name VARCHAR(50) NOT NULL,
    cashier_name VARCHAR(50),
    year INT,
    albums_purchased TEXT[] NOT NULL
)
"""
cur.execute(query)

In [8]:
query = """
INSERT INTO music_store
(transaction_id, customer_name, cashier_name, year, albums_purchased)
VALUES
(%s, %s, %s, %s, %s)
"""

cur.execute(query, (1, "Amanda", "Sam", 2000, ["Rubber Soul", "Let it Be"]))
cur.execute(query, (2, "Toby", "Sam", 2000, ["My Generation"]))
cur.execute(query, (3, "Max", "Bob", 2018, ["Meet the Beatles", "Help!"]))

In [10]:
query = "SELECT * FROM music_store"
cur.execute(query)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Amanda', 'Sam', 2000, ['Rubber Soul', 'Let it Be'])
(2, 'Toby', 'Sam', 2000, ['My Generation'])
(3, 'Max', 'Bob', 2018, ['Meet the Beatles', 'Help!'])


### Moving to the First Normal Form

In [12]:
q1 = """
DROP TABLE IF EXISTS music_store
"""

q2 = """
CREATE TABLE IF NOT EXISTS music_store (
    transaction_id INT NOT NULL,
    customer_name VARCHAR(50) NOT NULL,
    cashier_name VARCHAR(50) NOT NULL,
    year INT,
    album_purchased VARCHAR(128) NOT NULL
)
"""

q3 = """
INSERT INTO music_store 
(transaction_id, customer_name, cashier_name, year, album_purchased)
VALUES
(%s, %s, %s, %s, %s)
"""

cur.execute(q1)
cur.execute(q2)
cur.execute(q3, (1, 'Amanda', 'Sam', 2000, 'Let it Be'))
cur.execute(q3, (1, 'Amanda', 'Sam', 2000, 'Rubber Soul'))
cur.execute(q3, (2, 'Toby', 'Sam', 2000, 'My Generation'))
cur.execute(q3, (3, 'Max', 'Bob', 2018, 'Help!'))
cur.execute(q3, (3, 'Max', 'Bob', 2018, 'Meet the Beatles'))
cur.execute('SELECT * FROM music_store')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Amanda', 'Sam', 2000, 'Let it Be')
(1, 'Amanda', 'Sam', 2000, 'Rubber Soul')
(2, 'Toby', 'Sam', 2000, 'My Generation')
(3, 'Max', 'Bob', 2018, 'Help!')
(3, 'Max', 'Bob', 2018, 'Meet the Beatles')


In [14]:
q1 = """
DROP TABLE IF EXISTS music_store
"""

q2 = """
CREATE TABLE IF NOT EXISTS music_store (
    transaction_id SERIAL PRIMARY KEY,
    customer_name VARCHAR(50) NOT NULL,
    cashier_name VARCHAR(50) NOT NULL,
    year INT,
    album_purchased VARCHAR(128) NOT NULL
)
"""

q3 = """
INSERT INTO music_store 
(customer_name, cashier_name, year, album_purchased)
VALUES
(%s, %s, %s, %s)
"""

cur.execute(q1)
cur.execute(q2)
cur.execute(q3, ('Amanda', 'Sam', 2000, 'Let it Be'))
cur.execute(q3, ('Amanda', 'Sam', 2000, 'Rubber Soul'))
cur.execute(q3, ('Toby', 'Sam', 2000, 'My Generation'))
cur.execute(q3, ('Max', 'Bob', 2018, 'Help!'))
cur.execute(q3, ('Max', 'Bob', 2018, 'Meet the Beatles'))
cur.execute('SELECT * FROM music_store')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Amanda', 'Sam', 2000, 'Let it Be')
(2, 'Amanda', 'Sam', 2000, 'Rubber Soul')
(3, 'Toby', 'Sam', 2000, 'My Generation')
(4, 'Max', 'Bob', 2018, 'Help!')
(5, 'Max', 'Bob', 2018, 'Meet the Beatles')


In [32]:
q1 = """
DROP TABLE IF EXISTS music_store, cashier, customer
"""

q2 = """
CREATE TABLE IF NOT EXISTS customer (
    customer_id SERIAL PRIMARY KEY,
    customer_name VARCHAR(50) NOT NULL
)
"""

q3 = """
CREATE TABLE IF NOT EXISTS cashier (
    cashier_id SERIAL PRIMARY KEY,
    cashier_name VARCHAR(50) NOT NULL
)
"""

q6 = """
INSERT INTO customer 
(customer_name)
VALUES
(%s)
"""

q5 = """
INSERT INTO cashier
(cashier_name)
VALUES
(%s)
"""

cur.execute(q1)
cur.execute(q2)
cur.execute(q3)

cur.execute(q5, ('Sam', ))
cur.execute(q5, ('Bob', ))
cur.execute(q6, ('Amanda', ))
cur.execute(q6, ('Toby', ))
cur.execute(q6, ('Max', ))

cur.execute('SELECT * FROM customer')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
print()
cur.execute('SELECT * FROM cashier')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Amanda')
(2, 'Toby')
(3, 'Max')

(1, 'Sam')
(2, 'Bob')


In [33]:
q4 = """
CREATE TABLE IF NOT EXISTS music_store (
    transaction_id SERIAL PRIMARY KEY,
    customer_id INT REFERENCES customer(customer_id),
    cashier_id INT REFERENCES cashier(cashier_id),
    year INT,
    album_purchased VARCHAR(128) NOT NULL
)
"""

q7 = """
INSERT INTO music_store 
(customer_id, cashier_id, year, album_purchased)
VALUES
(%s, %s, %s, %s)
"""

cur.execute(q4)
cur.execute(q7, (1, 1, 2000, 'Let it Be'))
cur.execute(q7, (1, 1, 2000, 'Rubber Soul'))
cur.execute(q7, (2, 1, 2000, 'My Generation'))
cur.execute(q7, (3, 2, 2018, 'Help!'))
cur.execute(q7, (3, 2, 2018, 'Meet the Beatles'))
cur.execute('SELECT * FROM music_store')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 1, 1, 2000, 'Let it Be')
(2, 1, 1, 2000, 'Rubber Soul')
(3, 2, 1, 2000, 'My Generation')
(4, 3, 2, 2018, 'Help!')
(5, 3, 2, 2018, 'Meet the Beatles')


In [38]:
join_query = """
SELECT transaction_id, album_purchased, year, customer_name, cashier_name FROM (music_store
JOIN cashier
ON cashier.cashier_id = music_store.cashier_id)
JOIN customer
ON customer.customer_id = music_store.customer_id
"""
cur.execute(join_query)
cur.fetchall()

[(1, 'Let it Be', 2000, 'Amanda', 'Sam'),
 (2, 'Rubber Soul', 2000, 'Amanda', 'Sam'),
 (3, 'My Generation', 2000, 'Toby', 'Sam'),
 (4, 'Help!', 2018, 'Max', 'Bob'),
 (5, 'Meet the Beatles', 2018, 'Max', 'Bob')]

In [42]:
cur.execute('DROP TABLE IF EXISTS music_store')
cur.execute('DROP TABLE IF EXISTS cashier')
cur.execute('DROP TABLE IF EXISTS customer')

In [43]:
cur.close()
conn.close()